In [1]:
import pandas as pd
import requests, csv, time, os
from typing import Optional, Dict
from functools import lru_cache  # CACHE

In [78]:
# Basis-Endpunkt der MediaWiki API (Wikidata)
API_ENDPOINT = "https://www.wikidata.org/w/api.php"

# HTTP-Header für API-Zugriffe, insbesondere ein benutzerdefinierter User-Agent (erforderlich laut API-Richtlinien)
HEADERS = {"User-Agent": "NFDI4Microbiota-QS-Generator/2.0 (info@example.com)"}

# Maximale Anzahl an Wiederholungsversuchen bei Fehlern (z. B. Rate-Limit, Timeout)
MAX_RETRIES = 3

# Wartezeit in Sekunden zwischen Wiederholungsversuchen (Back-off-Zeit)
BACKOFF_SECS = 3

In [79]:
"""
Führt eine GET-Anfrage an die Wikidata API aus, mit Fehlerbehandlung und automatischer Wiederholung.
"""

def _api_get(params: Dict) -> Dict:  # API
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            # Führe GET-Anfrage aus, mit Header und Timeout
            r = requests.get(API_ENDPOINT, params=params, headers=HEADERS, timeout=25)

            # Löst eine Ausnahme bei HTTP-Fehlern aus (z. B. 403, 500)
            r.raise_for_status()

            # JSON-Antwort dekodieren und zurückgeben
            return r.json()

        except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError):
            # Bei Timeout oder Verbindungsfehler: warte (linear steigend) und versuche es erneut
            wait = BACKOFF_SECS * attempt
            print(f"[warn] API timeout – Versuch {attempt}/{MAX_RETRIES}, warte {wait}s …")
            time.sleep(wait)

        except requests.exceptions.HTTPError as e:
            # Bei HTTP-Fehlern (z. B. 403, 500): abbrechen, nicht erneut versuchen
            print(f"[error] API HTTP {e.response.status_code}: {e.response.reason}")
            break
    # Rückgabe bei Fehlschlag aller Versuche: leeres Dictionary
    return {}

In [80]:
"""
Findet die Wikidata-QID zu einer gegebenen ORCID-ID mithilfe der Suchfunktion 'haswbstatement'.
"""

# Aktiviere Caching, um doppelte API-Anfragen zu vermeiden
@lru_cache(maxsize=None)
def find_qid_by_orcid(orcid: str) -> Optional[str]:
    # Leere Eingabe sofort abbrechen
    if not orcid:
        return None
    # Baue die spezielle Suchanfrage (CirrusSearch über ORCID property P496)
    query = f'haswbstatement:P496="{orcid}"'

    # Anfrage an Wikidata-API stellen
    data = _api_get(
        {
            "action": "query",
            "list": "search",
            "srsearch": query,
            "srlimit": 1,
            "format": "json"
        }
    )
    try:
        # Extrahiere QID aus dem ersten Suchtreffer (z. B. "Q12345")
        return data["query"]["search"][0]["title"]
    except (KeyError, IndexError):
        # Kein Treffer gefunden oder Antwort unerwartet → gib None zurück
        return None

In [81]:
"""
Sucht die Wikidata-QID für ein gegebenes Label (Name), optional sprachspezifisch.
"""

@lru_cache(maxsize=None)  # API/cache
def find_qid_by_name(name: str, lang: str = "en") -> Optional[str]:
    # Leere Eingabe direkt abbrechen
    if not name:
        return None

    # Sende API-Anfrage an den wbsearchentities-Endpunkt (Wikidata-Suche)
    data = _api_get(
        {
            "action": "wbsearchentities",
            "search": name,
            "language": lang,
            "type": "item",
            "limit": 1,
            "format": "json"
        }
    )
    try:
        # Gib die Q-ID des ersten Treffers zurück (z. B. "Q123456")
        return data["search"][0]["id"]
    except (KeyError, IndexError):
        # Kein Treffer oder unvollständige Antwort → gib None zurück
        return None

In [82]:
"""
Sucht nach einer Wikidata-QID für eine Institution anhand ihres Labels.
Ergebnisse werden im lokalen Cache gespeichert.
"""

# Einfacher Cache für Institutionslabels → QID (oder None, falls nicht gefunden)
inst_cache: Dict[str, Optional[str]] = {}

def find_qid_by_institution_label(label: str) -> Optional[str]:  # API
    # Falls keine Eingabe → abbrechen
    if not label:
        return None

    # Falls Ergebnis schon im Cache → direkt zurückgeben
    if label in inst_cache:
        return inst_cache[label]

    # Durchsuche Wikidata nach dem Label – zuerst auf Englisch, dann auf Deutsch
    for lang in ("en", "de"):
        data = _api_get({
            "action": "wbsearchentities", "search": label, "language": lang,
            "type": "item", "limit": 1, "format": "json"})

        # Falls Treffer vorhanden → QID extrahieren und cachen
        if data.get("search"):
            qid = data["search"][0]["id"]
            inst_cache[label] = qid

            # Optionale Ausgabe, wenn deutsches Label verwendet wurde
            if lang == "de":
                print(f"[info] Institution '{label}' über deutsches Label gefunden → {qid}")
            return qid

    # Kein Treffer in beiden Sprachen → Cache mit None füllen
    inst_cache[label] = None
    return None

In [83]:
def file_to_qs(infile: str, outfile: str) -> None:
    # Bestimme Dateierweiterung (xls/xlsx oder csv)
    ext = os.path.splitext(infile)[1].lower()

    # Lese die Eingabedatei je nach Format ein
    df = pd.read_excel(infile) if ext in {".xlsx", ".xls"} else pd.read_csv(infile)

    # Prüfe, ob alle erforderlichen Spalten vorhanden sind
    required = {"Name", "Institution", "ORCID", "ORCID-Link"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Fehlende Spalten: {', '.join(sorted(missing))}")

    # Initialisiere Ergebnisliste und Duplikat-Tracker
    rows = []
    processed = set()

    # Iteriere über alle Zeilen der Eingabedatei
    for _, r in df.iterrows():
        name = str(r["Name"]).strip()

        # ORCID ggf. leer setzen, wenn NaN
        orcid = str(r["ORCID"]).strip() if pd.notna(r["ORCID"]) else ""

        # Dedupliziere anhand von Name + ORCID (kleingeschrieben)
        key = (name.lower(), orcid)
        if key in processed:
            continue
        processed.add(key)

        # Institution und URL vorbereiten
        inst_label = str(r["Institution"]).strip()
        url = r["ORCID-Link"] if pd.notna(r["ORCID-Link"]) else ""

        # Prüfe, ob Person bereits existiert (über ORCID oder Name)
        qid = find_qid_by_orcid(orcid) or find_qid_by_name(name)
        if qid:
            print(f"[skip] {name} existiert bereits als {qid}")
            continue

        # Versuche, die QID der Institution zu finden
        inst_qid = find_qid_by_institution_label(inst_label)
        if not inst_qid:
            print(f"[warn] Institution '{inst_label}' nicht gefunden ⇒ übersprungen")
            continue

        # Baue QuickStatements-Zeile
        rows.append({
            "qid": "",
            "Len": name,
            "P31": "Q5",          # instance of → human
            "P496": orcid,        # ORCID
            "S854": url,          # Quelle (URL)
            "P108": inst_qid,     # employer/affiliation
        })

        # Kurze Pause, um API nicht zu überlasten
        time.sleep(0.1)

    # Falls keine neuen Zeilen → keine Ausgabe
    if not rows:
        print("Keine neuen Items – nichts exportiert.")
        return

    # Schreibe die QuickStatements-Datei im CSV-Format
    field_order = ["qid", "Len", "P31", "P496", "S854", "P108"]
    with open(outfile, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=field_order)
        writer.writeheader()
        writer.writerows(rows)

    # Erfolgsmeldung mit Zeilenanzahl
    print(f"✓ {len(rows)} QuickStatements-Zeilen → {outfile}")

In [84]:
# Pfad zur Eingabedatei mit Personen, Institutionen und ORCID-Informationen
csv_input_path = "import/input_with_orcid.csv"

# Pfad zur Zieldatei für generierte QuickStatements im CSV-Format
csv_output_path = "import/quickstatements.csv"

# Starte die Verarbeitung: prüfe vorhandene QIDs und erstelle neue QS-Zeilen
file_to_qs(csv_input_path, csv_output_path)

[skip] Alexander Sczyrba existiert bereits als Q30420936
[skip] Jens Stoye existiert bereits als Q89498719
[skip] Michael Beckstette existiert bereits als Q114411617
[skip] Liren Huang existiert bereits als Q114780829
[skip] Sebastian Jünemann existiert bereits als Q56948964
[skip] Kassian Kobert existiert bereits als Q133094637
[skip] Anandhi Iyappan existiert bereits als Q59196905
[skip] Peer Bork existiert bereits als Q7160367
[skip] Sarah Schulz existiert bereits als Q65162179
[skip] Daniel Podlesny existiert bereits als Q133331882
[skip] Manja Marz existiert bereits als Q87730329
[skip] Winfried Göttsch existiert bereits als Q44200631
[skip] Anderson Santos existiert bereits als Q39510481
[skip] Ulisses Nunes da Rocha existiert bereits als Q47007256
[skip] Martin Bole existiert bereits als Q102304978
[skip] Adrian Fritz existiert bereits als Q133333363
[skip] Alice McHardy existiert bereits als Q2646932
[skip] Mattea Müller existiert bereits als Q56957915
[skip] Fernando Meyer exi